# Training Dialogue Encoder

## Data Validation

In [1]:
from train_utils import ContrastiveDataset


dataset = ContrastiveDataset('/home/alekseev_ilya/dialogue-augmentation/dialogue_encoder/dataset/train')

In [2]:
len(dataset)

40957

In [6]:
from tqdm import tqdm
for i in tqdm(range(len(dataset))):
    _ = dataset[i]

  0%|          | 121/40957 [00:07<40:13, 16.92it/s]


KeyboardInterrupt: 

## Model

In [ ]:
import os
import json


n_chunks = 80
chunks_original = sorted([filename for filename in os.listdir(path_original) if filename.endswith('.json')])[:n_chunks]
chunks_positive = sorted([filename for filename in os.listdir(path_positive) if filename.endswith('.json')])[:n_chunks]

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
torch.set_float32_matmul_precision('medium')    

In [ ]:
from nup.models.dialogue import SimpleDialogueEncoder


encoder_name = 'xlnet-base-cased'
model = SimpleDialogueEncoder(encoder_name)

In [ ]:
# from nup.models.dialogue import HSSAConfig, HSSADM
# config = HSSAConfig(
#     max_ut_embeddings=28,
#     casual_utterance_attention=False,
# )
# dialogue_model = HSSADM(
#     'microsoft/mpnet-base',
#     config,
#     pool_utterance_level=False,
#     pool_dialogue_level=True
# ).cuda()
# dialogue_model.requires_grad_(False)